In [1]:
import numpy as np

# Define the dimensions of each layer in ResNet50
layers = {
    'input': (224, 224, 3),        # Input layer
    'zeroPad': (230, 230, 3),      # Zero Padding
    'conv2D': (112, 112, 64),      # Initial Conv2D
    'maxpool': (55, 55, 64),       # Max Pooling
    'stage_1_A': (55, 55, 256),    # Stage A - Block 1
    'stage_2_B': (28, 28, 512),    # Stage B - Block 2
    'stage_3_C': (14, 14, 1024),   # Stage C - Block 3
    'stage_4_D': (7, 7, 2048),     # Stage D - Block 4
    'avgpool': (1, 1, 2048),       # Average Pooling
    'fc': (1, 1, 1000)             # Fully Connected Layer
}

# Define the kernel size and number of parameters for convolutional layers
conv_kernels = {
    'conv2D': (7, 7, 3, 64),               # Conv2D: (kernel height, kernel width, in_channels, out_channels)
    'stage_1_A': (3, 3, 64, 256),          # Stage A: (3x3 Conv, 64->256)
    'stage_2_B': (3, 3, 256, 512),         # Stage B: (3x3 Conv, 256->512)
    'stage_3_C': (3, 3, 512, 1024),        # Stage C: (3x3 Conv, 512->1024)
    'stage_4_D': (3, 3, 1024, 2048)        # Stage D: (3x3 Conv, 1024->2048)
}

# Function to compute memory in bytes for a given tensor shape
def compute_memory(tensor_shape, dtype=np.float32):
    element_size = np.dtype(dtype).itemsize  # Size of each element in bytes (default: float32 = 4 bytes)
    return np.prod(tensor_shape) * element_size  # Total memory in bytes

# Calculate the peak memory usage for each layer in bytes
def peak_memory_profiling(layers, conv_kernels):
    memory_profile = {}
    
    for layer_name, shape in layers.items():
        if layer_name in conv_kernels:  # Convolutional Layer
            # Get kernel parameters: (kernel height, kernel width, in_channels, out_channels)
            k_h, k_w, in_ch, out_ch = conv_kernels[layer_name]

            # Calculate memory for activations
            activation_memory = compute_memory(shape)

            # Calculate memory for weights (height * width * in_channels * out_channels)
            weights_memory = compute_memory((k_h, k_w, in_ch, out_ch))
            
            # Calculate memory for biases (optional, usually 1 per output channel)
            bias_memory = compute_memory((out_ch,))

            # Total memory
            peak_memory = activation_memory + weights_memory + bias_memory

        elif layer_name == 'fc':  # Fully Connected Layer
            # Fully Connected: Consider weights and output size
            input_features = np.prod(layers['avgpool'])  # Input size to FC from the previous avgpool layer
            output_features = shape[2]  # Number of neurons in the FC layer
            weights_memory = compute_memory((input_features, output_features))  # Weights
            bias_memory = compute_memory((output_features,))  # Bias memory
            activation_memory = compute_memory(shape)
            
            peak_memory = activation_memory + weights_memory + bias_memory

        else:  # For non-convolutional layers (like pooling)
            peak_memory = compute_memory(shape)
        
        memory_profile[layer_name] = peak_memory
    
    return memory_profile

# Run the profiling function
memory_usage = peak_memory_profiling(layers, conv_kernels)

# Convert to human-readable format and print results
for layer, memory in memory_usage.items():
    print(f"Layer: {layer}, Peak Memory Usage: {memory / (1024 ** 2):.2f} MB")


Layer: input, Peak Memory Usage: 0.57 MB
Layer: zeroPad, Peak Memory Usage: 0.61 MB
Layer: conv2D, Peak Memory Usage: 3.10 MB
Layer: maxpool, Peak Memory Usage: 0.74 MB
Layer: stage_1_A, Peak Memory Usage: 3.52 MB
Layer: stage_2_B, Peak Memory Usage: 6.03 MB
Layer: stage_3_C, Peak Memory Usage: 18.77 MB
Layer: stage_4_D, Peak Memory Usage: 72.39 MB
Layer: avgpool, Peak Memory Usage: 0.01 MB
Layer: fc, Peak Memory Usage: 7.82 MB
